## comments
position encoding -> 
    # if train_len is None:
    #     pos_list = np.arange(seq_len)
    # else:
    #     pos_list = np.linspace(0, train_len-1, num=seq_len)
    pos_list = np.arange(seq_len)

encoding layer -> 2 layers instead of 3

The model failed to align BECAUSE not training enough, check transformer_l2 for more

In [1]:
import sys
sys.path.append("/home/c1l1mo/projects/VideoAlignment")
sys.path.append("/home/c1l1mo/testings/carl")
sys.path.append("/home/c1l1mo/testings/carl/utils")
sys.path.append("/home/c1l1mo/testings/carl/datasets")
print(sys.path)

['/home/c1l1mo/projects/VideoAlignment/notebooks', '/home/c1l1mo/miniconda3/envs/carl/lib/python37.zip', '/home/c1l1mo/miniconda3/envs/carl/lib/python3.7', '/home/c1l1mo/miniconda3/envs/carl/lib/python3.7/lib-dynload', '', '/home/c1l1mo/.local/lib/python3.7/site-packages', '/home/c1l1mo/testings/HIT', '/home/c1l1mo/testings/pyskl', '/home/c1l1mo/miniconda3/envs/carl/lib/python3.7/site-packages', '/home/c1l1mo/.local/lib/python3.7/site-packages/IPython/extensions', '/home/c1l1mo/.ipython', '/home/c1l1mo/projects/VideoAlignment', '/home/c1l1mo/testings/carl', '/home/c1l1mo/testings/carl/utils', '/home/c1l1mo/testings/carl/datasets']


In [2]:
from model.transformer.transformer import CARL as Mine_Transformer
from models.transformer import TransformerModel as CARL_Transformer

/home/c1l1mo/.local/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import yaml
from easydict import EasyDict as edict
with open("/home/c1l1mo/projects/VideoAlignment/result/scl_penn_action/config.yml","r") as file:
    cfg = yaml.safe_load(file)
cfg = edict(cfg)

In [4]:
import numpy as np
import torch
import random

def setup_seed(seed):
    random.seed(seed)                          
    np.random.seed(seed)                       
    torch.manual_seed(seed)                    
    torch.cuda.manual_seed(seed)               
    torch.cuda.manual_seed_all(seed)           
    torch.backends.cudnn.deterministic = True  


setup_seed(7)

cfg.MODEL.PROJECTION = False
cfg.MODEL.L2_NORMALIZE= False
cfg.MODEL.EMBEDDER_MODEL.NUM_LAYERS=2

mine = Mine_Transformer(cfg)

setup_seed(7)
carl = CARL_Transformer(cfg)

/home/c1l1mo/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/c1l1mo/.local/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
carl

TransformerModel(
  (backbone): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): C

In [5]:
test = torch.rand(1,14,3,224,224)
test.shape

torch.Size([1, 14, 3, 224, 224])

In [6]:
setup_seed(7)
m = mine(test)

In [7]:
setup_seed(7)
c = carl(test)

In [8]:
torch.equal(m,c)

True

In [9]:
for mn,cn in zip(m,c):
    print(f'{mn}: ' , m[mn].shape)
    print(f'{cn}: ' , c[cn].shape)
    print(torch.equal(m[mn],c[cn]))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 for mn,cn in zip(m,c):                                                                       │
│ ❱ 2 │   print(f'{mn}: ' , m[mn].shape)                                                           │
│   3 │   print(f'{cn}: ' , c[cn].shape)                                                           │
│   4 │   print(torch.equal(m[mn],c[cn]))                                                          │
│   5                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: tensors used as indices must be long, byte or bool tensors

In [ ]:
x_pos = m['x_pos']
x_pos.shape

In [ ]:
mine.encoder.encodingLayers[1](x_pos,None)

In [ ]:
carl.embed.video_encoder.enc_layers[1](x_pos)

In [ ]:
from model.transformer.encoder.encodingLayer.encodingLayer import EncodingLayer

In [ ]:
setup_seed(7)
a = EncodingLayer(256,0.0,False)
setup_seed(7)
b = EncodingLayer(256,0.0,False)

In [ ]:
a.eval()
b.eval()
A = a(x_pos,None)
B = b(x_pos,None)

torch.equal(A,B)

In [ ]:
from copy import deepcopy
import torch.nn as nn
mineencodingLayers = nn.ModuleList([deepcopy(EncodingLayer(256,dout_p=0.0,test=False)) for _ in range(cfg.MODEL.EMBEDDER_MODEL.NUM_LAYERS)])

In [ ]:
mineencodingLayers.eval()
mineencodingLayers[0](x_pos,None)

In [ ]:
mineencodingLayers[1](x_pos,None)

In [ ]:
carl.embed.video_encoder.enc_layers[0].self_att.H

In [ ]:
from models.transformer import EncoderLayer
setup_seed(7)
encoderLayer = EncoderLayer(256,0.0,cfg.MODEL.EMBEDDER_MODEL.NUM_HEADS,cfg.MODEL.EMBEDDER_MODEL.D_FF, None)

In [ ]:
encoderLayer.eval()
ce = encoderLayer(x_pos,None)

torch.equal(ce,A)

In [ ]:
carlencodingLayers = nn.ModuleList([deepcopy(EncoderLayer(256,0.0,cfg.MODEL.EMBEDDER_MODEL.NUM_HEADS,cfg.MODEL.EMBEDDER_MODEL.D_FF, None)) for _ in range(cfg.MODEL.EMBEDDER_MODEL.NUM_LAYERS)])

In [ ]:
carlencodingLayers.eval()
carlencodingLayers[0](x_pos)

In [ ]:
carlencodingLayers[1](x_pos)

In [ ]:
from models.transformer import clone
Cloned = clone(EncoderLayer(256,0.0,cfg.MODEL.EMBEDDER_MODEL.NUM_HEADS,cfg.MODEL.EMBEDDER_MODEL.D_FF, None), cfg.MODEL.EMBEDDER_MODEL.NUM_LAYERS)

In [ ]:
Cloned.eval()
Cloned[0](x_pos)

In [ ]:
Cloned[1](x_pos)

In [ ]:
def mine_clone(m):
    return nn.ModuleList([deepcopy(m) for _ in range(cfg.MODEL.EMBEDDER_MODEL.NUM_LAYERS)])

In [ ]:
mineCloned = mine_clone(EncoderLayer(256,0.0,cfg.MODEL.EMBEDDER_MODEL.NUM_HEADS,cfg.MODEL.EMBEDDER_MODEL.D_FF, None))

In [ ]:
mineCloned.eval()
mineCloned[0](x_pos)

In [ ]:
mineCloned[1](x_pos)